<h1>Sieci neuronowe - ćwiczenie 3</h1>

In [1]:
!pip install ucimlrepo


[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [3]:
def sigmoid(n: int) -> float:
    return 1 / (1 + np.exp(-n))

In [264]:
def sigmoid_der(n: int) -> float:
    return sigmoid(n) * (1 - sigmoid(n))

In [266]:
class MultilayerNetwork:
   _weights: list[list[np.ndarray[float]]]
   _biases: list[np.ndarray[float]] 
   _caches_x: list[list[np.ndarray[float]]] 

   def __cross_entropy_loss(self, y: np.ndarray, y_pred:np.ndarray) -> np.ndarray:
      return np.sum(-y*np.log(y_pred) - (1 - y)*np.log(1 - y_pred)) / y_pred.shape[0]  
   
   def __cross_entropy_loss_der(self, y: np.ndarray, y_pred:np.ndarray) -> np.ndarray:
      return np.sum(((1 - y) / (1 - y_pred)) - (y/y_pred)) 

   def __init__(self, hidden_layers_sizes = (1,) ):
      self._weights = []
      self._biases = []
      self._caches_x = []

      for index, layer_size in enumerate(hidden_layers_sizes):
         self._weights.append([])
         self._biases.append([])
         self._caches_x.append([])
         for _ in range(layer_size):
            self._weights[index].append([])
            self._caches_x[index].append([])
      
      # output layer
      self._weights.append([])
      self._caches_x.append([])
      self._biases.append([])
      self._weights[-1].append([])
      self._caches_x[-1].append([])
   
   def __init_weights(self, size: int) -> np.ndarray:
        return np.random.rand(size, 1) # initialize weights randomly
   
   def __optimize(self, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray, batch_size: int, learning_rate: float, 
                       min_step: float, max_iter: int) -> (list, list, list, list, list, list, list, list):
      losses = []
      weights = []
      biases = []
      losses_test = []

      accuracy = []
      f_score = []
      precision = []
      recalls = []

      if batch_size > x_train.shape[0]:
         batch_size = x_train.shape[0]

      for index in range(max_iter): #learn for max_iter      
         old_train_loss = self.__cross_entropy_loss(self._weights, x_train, y_train, self._bias)

         shuffle = np.random.permutation(x_train.shape[0])
         x_train_shuffled = x_train[shuffle]
         y_train_shuffled = y_train[shuffle]

         for batch_start_index in range(0, x_train.shape[0], batch_size):
            x_train_batch = x_train_shuffled[batch_start_index:batch_start_index+batch_size] 
            y_train_batch = y_train_shuffled[batch_start_index:batch_start_index+batch_size] 

            pred = self.predict(x_train_batch)
            errors = self.backward(y_train_batch, pred)

            for layer_index, layer in enumerate(self._weights):
               for neuron_index, _ in enumerate(layer):
                  self._weights[layer_index][neuron_index] = self._weights[layer_index][neuron_index]  - learning_rate * np.dot(self._caches_x[layer_index][neuron_index], errors[layer_index][neuron_index])
                  self._biases[layer_index][neuron_index] = self._biases[layer_index][neuron_index] - learning_rate * np.sum(errors[layer_index][neuron_index])
                  

         #calculate loss of training and testing data
         new_train_loss = self.__cross_entropy_loss(y_train_batch, pred)
         new_test_loss = self.__cross_entropy_loss(self._weights, x_test, y_test, self._bias)

         #append to helper lists
         losses.append(new_train_loss)
         weights.append(self._weights)
         biases.append(self._bias)
         losses_test.append(new_test_loss)

         #calculate scores for each iteration
         y_pred = self.predict(x_test)
         accuracy.append(metrics.accuracy_score(y_test, y_pred))
         f_score.append(metrics.f1_score(y_test, y_pred))
         precision.append(metrics.precision_score(y_test, y_pred))
         recalls.append(metrics.recall_score(y_test, y_pred))

         if min_step is not None and index > 0 and abs(old_train_loss - new_train_loss) <= min_step: #if change of loss is smaller or equal than min_step when stop learning
               break

      return losses, weights, biases, losses_test, accuracy, f_score, precision, recalls

   def logarithmicRegression(self, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray, batch_size: int, learning_rate: float, 
                       max_iter: int, min_step: float = None) -> (list, list, list, list, list, list, list, list): #model convergence criteria are min_step (loss step) and max_iter (amount of iterations)
      
      curr_size_layer = x_train.shape[1]
      
      for layer_index, layer in enumerate(self._weights):
         for neuron_index, neuron in enumerate(layer):
            self._weights[layer_index][neuron_index] = self.__init_weights(curr_size_layer)
         curr_size_layer = len(layer)
         self._biases[layer_index] = self.__init_weights(curr_size_layer) #initialize bias randomly
      
      #fix shape of y data to match further calculations
      if len(y_test.shape) == 1:
         y_test = y_test[np.newaxis].T 

      if len(y_train.shape) == 1:
         y_train = y_train[np.newaxis].T

      losses, weights, biases, losses_test, accuracy, f_score, precision, recalls = self.__optimize(x_train, y_train, x_test, y_test, batch_size, learning_rate, min_step, max_iter) #optimize weights and bias using training data

      #plot results
      plt.plot(np.arange(len(losses)), losses, label="Train Loss")
      plt.plot(np.arange(len(losses)), losses_test, label="Test loss")
      plt.plot(np.arange(len(losses)), accuracy, label="Accuracy")
      plt.plot(np.arange(len(losses)), f_score, label="F_score")
      plt.plot(np.arange(len(losses)), precision, label="Precision")
      plt.plot(np.arange(len(losses)), recalls, label="Recall")
      plt.legend()
      plt.show()

      #print final weight, bias, loss and scores
      print("Weights: ", self._weights)
      print("Bias: ", self._bias)
      print("Train loss: ", losses[-1])
      print("Test loss: ", losses_test[-1])

      print("Scores")
      print("Accuracy: ", accuracy[-1])
      print("F_score: ", f_score[-1])
      print("Precision: ", precision[-1])
      print("Recall: ", recalls[-1])

      return losses, weights, biases, losses_test, accuracy, f_score, precision, recalls      
      

   def forward(self, x: np.ndarray) -> float:
      for layer_index, layer in enumerate(self._weights):
         for neuron_index, neuron in enumerate(layer):
            self._caches_x[layer_index][neuron_index] = x
         x = np.array([sigmoid(np.dot(x, neuron) + self._biases[layer_index][neuron_index]).flatten() for neuron in layer]).T
      return x

   def backward(self, y_train: np.ndarray, y_pred: np.ndarray) -> list[list[np.ndarray[float]]]:
      errors: list[list[np.ndarray[float]]] = []
      for layer_index, layer in enumerate(self._weights):
         errors.append([])
         for _ in layer:
            errors[layer_index].append([])

      err = self.__cross_entropy_loss_der(y_train, y_pred) * sigmoid_der(np.dot(self._caches_x[-1][0], self._weights[-1][0]) + self._biases[-1][0])
      errors[-1][0] = err
      next_weights = np.array(self._weights[-1]) # weights of last layer

      for layer_index, layer in reversed(enumerate(self._weights[:-1])):
         for neuron_index, _ in enumerate(layer):
            err = np.dot(err, next_weights)
            err = err * sigmoid_der(np.dot(self._caches_x[layer_index][neuron_index], self._weights[layer_index][neuron_index]) + self._biases[layer_index][neuron_index])
            errors[layer_index][neuron_index] = err
         next_weights = np.array(layer)
         
      return errors

   def predict(self, x_test: np.ndarray) -> np.ndarray:
        
      y_pred = self.forward(x_test) #calculate sigmoid for input data

      #change posibility from sigmoid to 1 or 0
      y_pred[y_pred >= 0.5] = 1
      y_pred[y_pred < 0.5] = 0
        
      return y_pred
    

<h3>Przygotowanie danych na podstawie poprzedniego ćwiczenia</h3>

In [8]:
# original code from https://archive.ics.uci.edu/dataset/45/heart+disease
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
heart_data = heart_disease.data.original

In [9]:
df: pd.DataFrame = heart_data

# repearing of the inbalnace in classification and removing null values
df["num"] = df["num"].replace([2, 3, 4], 1) #change classes to binary classification
print(df["num"].value_counts())

#get null values of ca and remove them
null_idx = df[df["ca"].isnull()].index 
print(null_idx)
df = df.drop(null_idx)
df = df.reset_index(drop=True) 
print(df["num"].value_counts())

#get null values of thel and remove them
null_idx = df[df["thal"].isnull()].index 
print(null_idx)
df = df.drop(null_idx)
df = df.reset_index(drop=True) 
print(df["num"].value_counts())

# balance classes to same amount 138
random_idx = df.query("num == 0").sample(df["num"].value_counts()[0] - df["num"].value_counts()[1]).index 
df = df.drop(random_idx)
df = df.reset_index(drop=True)
print(df["num"].value_counts())

df_without_num = df.loc[:, df.columns != "num"]
std_features = (df_without_num - df_without_num.mean() )/ df_without_num.std() #(value-mean)/variance

result = std_features
result["heart_disease"] = df["num"]

result

num
0    164
1    139
Name: count, dtype: int64
Index([166, 192, 287, 302], dtype='int64')
num
0    161
1    138
Name: count, dtype: int64
Index([87, 264], dtype='int64')
num
0    160
1    137
Name: count, dtype: int64
num
1    137
0    137
Name: count, dtype: int64


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heart_disease
0,1.378115,0.692333,0.856224,1.586864,0.794188,-0.412694,1.027809,-1.766472,1.392528,0.354756,0.630769,2.384985,-0.921154,1
1,1.378115,0.692333,0.856224,-0.659298,-0.366440,-0.412694,1.027809,-0.848538,1.392528,1.295154,0.630769,1.337732,1.135027,1
2,-1.568371,-1.439120,-1.238467,-0.097757,-0.875487,-0.412694,1.027809,1.031040,-0.715498,0.269265,-1.015237,-0.756774,-0.921154,0
3,0.131525,0.692333,-1.238467,-0.659298,-0.223907,-0.412694,-0.983760,1.293307,-0.715498,-0.243680,-1.015237,-0.756774,-0.921154,0
4,0.811483,-1.439120,0.856224,0.463783,0.427674,-0.412694,1.027809,0.506507,-0.715498,2.150062,2.276774,1.337732,-0.921154,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,0.244851,-1.439120,0.856224,0.463783,-0.122097,-0.412694,-0.983760,-1.110805,1.392528,-0.756625,0.630769,-0.756774,1.135027,1
270,-1.115065,0.692333,-2.285812,-1.220838,0.346227,-0.412694,-0.983760,-0.717405,-0.715498,0.098283,0.630769,-0.756774,1.135027,1
271,1.491441,0.692333,0.856224,0.688399,-1.099468,2.414260,-0.983760,-0.324005,-0.715498,1.979081,0.630769,1.337732,1.135027,1
272,0.244851,0.692333,0.856224,-0.097757,-2.361906,-0.412694,-0.983760,-1.460494,1.392528,0.098283,0.630769,0.290479,1.135027,1


In [10]:
def train_test_split(features, targets, percentage):
    choices = np.random.choice(range(features.shape[0]), size=(int(features.shape[0] * percentage/100),), replace=False) 
    split = np.zeros(features.shape[0], dtype=bool)
    split[choices] = True

    return features[split], targets[split], features[~split], targets[~split]

In [125]:
features = result.loc[:, result.columns != "heart_disease"].to_numpy()
targets = result["heart_disease"].to_numpy()

x_train, y_train, x_test, y_test = train_test_split(features, targets, 70)

In [263]:
max_iter = 200

ex1 = MultilayerNetwork((3,2))
result_ex1 = ex1.logarithmicRegression(x_train, y_train, x_test, y_test, 0.2, max_iter, 0.0001)
ex1.predict(x_test[63])

[[array([-2.36165554, -1.43911965, -1.23846671, -0.77160578, -0.75331594,
       -0.41269396, -0.98376047,  1.90526254, -0.71549768, -0.32917077,
       -1.01523693, -0.75677419, -0.92115385]), array([-2.36165554, -1.43911965, -1.23846671, -0.77160578, -0.75331594,
       -0.41269396, -0.98376047,  1.90526254, -0.71549768, -0.32917077,
       -1.01523693, -0.75677419, -0.92115385]), array([-2.36165554, -1.43911965, -1.23846671, -0.77160578, -0.75331594,
       -0.41269396, -0.98376047,  1.90526254, -0.71549768, -0.32917077,
       -1.01523693, -0.75677419, -0.92115385])], [array([[0.01492239, 0.02781377, 0.01535772]]), array([[0.01492239, 0.02781377, 0.01535772]])], [array([[0.56563456, 0.56490518]])]]


array([[1.]])

In [287]:
shuffle = np.random.permutation(y_train.shape[0])

y_train[np.newaxis].T[shuffle].shape


(191, 1)